In [1]:
import os

# Restrict PyTorch to only see GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import torch

if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available, using CPU.")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Number of GPUs available: 1
GPU 0: NVIDIA L40S
Using device: cuda:0


In [2]:
import tqdm
import tensorflow as tf
import torch
import wandb
import os
import copy
import json
import logging
import pathlib
from tqdm import tqdm
import transformers
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_scheduler
from transformers.trainer import ALL_LAYERNORM_LAYERS, get_parameter_names
from PIL import Image
from accelerate import PartialState
from dataclasses import dataclass, asdict
from typing import Dict, Optional, Sequence, List
from torch.nn import MSELoss, L1Loss, SmoothL1Loss
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast

from spatialvla.datasets import RLDSDataset, RLDSBatchTransform
from spatialvla.datasets.rlds.utils.data_utils import save_dataset_statistics
from spatialvla.datasets.rlds.utils.data_utils import PaddedCollatorForActionPrediction

from spatialvla.mobilevlm.model.mobilevlm import load_pretrained_vlm_for_vla
from spatialvla.mobilevlm.train.train import find_all_linear_names, get_peft_state_maybe_zero_3, get_peft_state_non_lora_maybe_zero_3, find_all_names_from_module

from scripts.spatialvla_config import ModelArguments, TrainingArguments, HEAD_ARGS

2024-11-29 23:08:52.096213: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 23:08:52.127152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 23:08:52.127202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 23:08:52.128243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 23:08:52.134136: I tensorflow/core/platform/cpu_feature_guar

[2024-11-29 23:08:54,596] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
model_args = ModelArguments()
model_args.action_head = 'DiT'
model_args.head_args = HEAD_ARGS['DiT']
training_args = TrainingArguments()
dtype = torch.bfloat16
device_id = 0

In [4]:
tokenizer, model, image_processor, _ = load_pretrained_vlm_for_vla(
    model_args, 
    load_8bit=False, 
    load_4bit=False,
    device=device_id,
    dtype=dtype
)

Loading with torch.bfloat16


/home/jellyho/miniconda3/envs/mobilevlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type mobilevlm to instantiate a model of type spatialvla. This is not supported for all configurations of models and can yield errors.
Some weights of SpatialVLAForCausalLM were not initialized from the model checkpoint at remyxai/SpaceLLaVA-lite and are newly initialized: ['action_head.time_net.out_net.2.weight', 'action_head.eps_net.adaLN_modulation.1.bias', 'action_head.action_proj.2.bias', 'action_head.action_proj.0.weight', 'action_head.eps_net.linear.weight', 'action_head.eps_net.adaLN_modulation.1.weight', 'action_head.time_net.out_net.0.weight', 'action_head.action_proj.0.bias', 'action_head.timestep_pos', 'action_head.noise_pos', 'action_head.ti

In [5]:
model = model.to(device_id)

In [6]:
for param in model.parameters():
    param.requires_grad = False

In [11]:
batch_transform = RLDSBatchTransform(
    tokenizer,
    image_processor,
)

dataset = RLDSDataset(
    data_root_dir=training_args.data_root_dir,
    data_mix=training_args.data_mix,
    batch_transform=batch_transform,
    shuffle_buffer_size=training_args.shuffle_buffer_size,
    train=True,
    window_size=1,
    future_action_window_size=model_args.action_len - 1
)
collator = PaddedCollatorForActionPrediction(tokenizer.model_max_length, tokenizer.pad_token_id, padding_side='right')

dataloader = DataLoader(
    dataset,
    batch_size=1,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)

2024-11-29 23:09:38.366583: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-29 23:09:38.482605: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization



######################################################################################
# Loading the following 1 datasets (incl. sampling weight):                         #
# libero_object_no_noops: ==================================================1.000000 #
######################################################################################

Threads per Dataset: %s [1]
Reads per Dataset: %s [1]
Constructing datasets...


2024-11-29 23:09:38.618366: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Applying frame transforms on dataset...


In [21]:
batch_idx, batch = next(enumerate(dataloader))
model = model.eval()

In [75]:
input_ids=batch['input_ids'].to(device_id)
images=batch['pixel_values'].to(device_id)
attention_mask=batch['attention_mask'].to(device_id)
actions=batch['action'].to(device_id)
past_key_values = None
labels = None

import time
with torch.autocast('cuda', dtype=dtype):
    torch.cuda.synchronize()
    start = time.time()
    # input_ids, attention_mask, past_key_values, inputs_embeds, labels = model.prepare_inputs_labels_for_multimodal(input_ids, attention_mask, past_key_values, labels, images)
    # input_ids, attention_mask, past_key_values, inputs_embeds, labels, time_enc, noise =  model.action_head.prepare_inputs_for_DiT_training(actions, input_ids, attention_mask, past_key_values, inputs_embeds, labels)
    outputs = model.predict_action(
            input_ids=batch['input_ids'].to(device_id),
            images=batch['pixel_values'].to(device_id),
            attention_mask=batch['attention_mask'].to(device_id),
            use_cache=True
        )
    torch.cuda.synchronize()
    end = time.time()
print(end - start)

0.16927671432495117


In [10]:
model_args

ModelArguments(model_path='remyxai/SpaceLLaVA-lite', action_head='DiT', head_args={'head_type': 'DiT', 'hidden_projection': 'pass', 'use_map': False, 'max_action': 5.0, 'time_dim': 256, 'hidden_dim': 256, 'diffusion_steps': 100, 'sched': 'DDIM'}, action_dim=7, action_len=1)